In [ ]:
!pip install face_recognition
!pip install opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=5e239a8bfa042033b336f9f3abeb05afd058b847a1c95ced553d16e13a85b412
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import os
import face_recognition
from google.colab import files
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import PIL.Image
import io
import time


In [ ]:
# Step 1: Upload known face images
os.makedirs("known_faces", exist_ok=True)  # Create a directory for known faces

# Prompt to upload known images
print("Please upload images of known individuals (e.g., name1.jpg, name2.jpg, etc.)")
uploaded = files.upload()

# Step 2: Process the uploaded known images
known_encodings = []
known_names = []

for filename in uploaded.keys():
    # Save uploaded file to the known_faces directory
    with open(f"known_faces/{filename}", "wb") as f:
        f.write(uploaded[filename])

    # Load and encode the face from the image
    image = face_recognition.load_image_file(f"known_faces/{filename}")
    encoding = face_recognition.face_encodings(image)[0]  # Encode the face

    # Add encoding and name to respective lists
    known_encodings.append(encoding)
    name = os.path.splitext(filename)[0]  # Extract name from filename
    known_names.append(name)

print(f"{len(known_encodings)} known faces uploaded and processed successfully!")


def capture_image():
    js = Javascript('''
        async function captureImage() {
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const context = canvas.getContext('2d');

            // Stream from webcam
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            // Capture frame
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            context.drawImage(video, 0, 0, canvas.width, canvas.height);

            // Stop video
            stream.getTracks().forEach(track => track.stop());

            // Return image as base64 string
            return canvas.toDataURL('image/png');
        }
        captureImage();
    ''')
    display(js)
    data = eval_js('captureImage()')
    return data


def process_image(image_data):
    header, encoded = image_data.split(',', 1)
    image_bytes = b64decode(encoded)
    pil_image = PIL.Image.open(io.BytesIO(image_bytes))

    # Convert the image to RGB format
    pil_image = pil_image.convert('RGB')

    # Convert to NumPy array
    image_np = np.array(pil_image)
    return image_np


Please upload images of known individuals (e.g., name1.jpg, name2.jpg, etc.)


Saving img (2).jpg to img (2) (1).jpg
1 known faces uploaded and processed successfully!


In [ ]:
# Step 4: Capture the test face (verification)
print("Now, please capture the test face for verification.")
image_data = capture_image()

# Process the captured image
test_image = process_image(image_data)

# Encode the face in the captured test image
test_encodings = face_recognition.face_encodings(test_image)


if not test_encodings:
    print("No face detected in the test image.")
else:
    test_encoding = test_encodings[0]

    # Compare with known faces
    matches = face_recognition.compare_faces(known_encodings, test_encoding)
    face_distances = face_recognition.face_distance(known_encodings, test_encoding)

    if any(matches):
        match_index = matches.index(True)
        name = known_names[match_index]
        print(f"Access Granted! Welcome, {name}.")
    else:
        print("Access Denied.")



Now, please capture the test face for verification.


<IPython.core.display.Javascript object>

Access Granted! Welcome, img (2) (1).
